In [1]:
from FS.constant import DEFAULT_NAME,DEFAULT_PATH,ALLOWED_TAGS_EXT
from FS.stack import Stack
from FS.prifixSearch import Trie
from FS.hash import HashMaster
from FS.extSearch import CustomHashMap
from FS.fileReader import universal_reader
from FS.useful import needs_rehash,file_hash,name_ext

# import subprocess
# from mrvector import MrVectorExpert
import os
import sys
import shutil
import hashlib
from pathlib import Path
import pickle
from queue import Queue


In [2]:


class Node:
    def __init__(self,name,path):
        self.path = path
        self.name = name
        self.childs = {}
        self.type = 'dir'

        self.state = "pending"

        self.prifixSearch = Trie()
        self.extensionSearch =  CustomHashMap()

        self.hash = None
        self.vector = None

        # st.st_size
        self.size = None
        # st.st_mtime
        self.modified_time = None
        # st.st_ctime
        self.created_time = None
        # st.st_mode
        self.mode = None

    def _config_stat(self,st):
        self.size = st.st_size
        self.modified_time = st.st_mtime
        self.created_time = st.st_ctime
        self.mode = st.st_mode
    
    def config(self,item):
        st = item.stat()
        self._config_stat(st)
        if item.is_dir():
            self.type = 'dir'
        else:
            self.type = 'file'
    def config_by_path(self,path):
        st = os.stat(path)
        self._config_stat(st)

    def _add_file_to_search_structure(self,node):
        self.prifixSearch.insert(node.name,node.path)
        self.extensionSearch.insert(node.name,node.path)
    
    def _del_file_to_search_structure(self,node):
        self.prifixSearch.delete(node.name,node.path)
        self.extensionSearch.delete(node.name,node.path)

    def _add_dir_to_search_structure(self,node):
        self.prifixSearch.insert(node.name,node.path)

    def _del_dir_to_search_structure(self,node):
        self.prifixSearch.delete(node.name,node.path)

    def search_prifix(self,prifix):
        return self.prifixSearch.search_by_prefix(prifix)
    
    def search_ext(self,ext):
        return self.extensionSearch.get_by_extension(ext)

    def to_dict(self):
        return {
            'name': self.name,
            'path': self.path,
            'type': self.type,
            'size': self.size,
            'modified_time': self.modified_time,
            'created_time': self.created_time,
            'mode': self.mode,
            'childs': [name for name in self.childs.keys()]
        }
    
    
    def __repr__(self):
        return f"Node(name={self.name}, path={self.path}, type={self.type})"




In [27]:


class FSManager:
    '''file management'''
    def __init__(self,tagG,db_path="save/root.pkl", autosave=True):
        self.default = DEFAULT_PATH
        self.default_name = DEFAULT_NAME
        self.tagG = tagG
        self.root = None
        self.root_store_path = db_path
        self.autosave = autosave
        # self.prifixSearch = Trie()
        # self.extensionSearch = CustomHashMap()
        self.cwd = self.root
        self.selected_nodes = []
        self.state = 'ideal'
        self.pointer = None
        self.history = Stack()
        self.hash_queue = []
        self.hash_Master = HashMaster()
        self.tag_queue = Queue()

    def normalize_path(self,path):
        return Path(path).as_posix()

    def save(self):
        """
        Save HashMaster state to pickle
        """
        data = {
            "root": self.root
        }
        with open(self.root_store_path, "wb") as f:
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    def save_if_autosave(self):
        if self.autosave:
            self.save()

    def load(self):
        """
        Load HashMaster state from pickle
        """
        if not os.path.exists(self.root_store_path):
            print("save first! file not exist!")
            self.root = Node(self.default_name,self.default)
            self.cwd = self.root
            self.refresh_root()
            return

        try:
            with open(self.root_store_path, "rb") as f:
                data = pickle.load(f)

            self.root = data.get("root", {})
            self.cwd = self.root

        except Exception:
            # Corrupted pickle → safe reset
            self.root = Node(self.default_name,self.default)
            self.cwd = self.root

    '''                                                                                
        ██████╗ ███████╗███████╗██████╗ ███████╗███████╗██╗  ██╗
        ██╔══██╗██╔════╝██╔════╝██╔══██╗██╔════╝██╔════╝██║  ██║
        ██████╔╝█████╗  █████╗  ██████╔╝█████╗  ███████╗███████║
        ██╔══██╗██╔══╝  ██╔══╝  ██╔══██╗██╔══╝  ╚════██║██╔══██║
        ██║  ██║███████╗██║     ██║  ██║███████╗███████║██║  ██║
        ╚═╝  ╚═╝╚══════╝╚═╝     ╚═╝  ╚═╝╚══════╝╚══════╝╚═╝  ╚═╝    
    '''
    def _refresh(self,node): # work good
        #>>>>>>>>>>>>>>>>>>WE NEED TO IMPLEMENT THE MODIFICATION TIME DETECTION AND AFTER THAT CONFIG NODE 
        #  CHECK FULL CODE AND ANALYIS WHERE THE CONFIG NODE IS NEEDED SO THE TREE BECOME UPTO DATE
        #  THINK ABOUT WE NEED HASH OR NOT FOR CHANGE DETECTION 
        # IMPLEMET PICKEL PART TO STORE HASHMAP GRAPH TIER 
        internal = list(node.childs.keys())
        orignal = list(os.scandir(node.path))
        orignal_lst = [f.name for f in orignal]
        #  create part
        for item in orignal:
            if item.name not in internal:
                # new file or folder added
                print("new file or folder detected:",item.name)
                newNode = Node(item.name,self.normalize_path(item.path))
                newNode.config(item)
                if item.is_dir():
                    node._add_dir_to_search_structure(newNode)
                else:
                    node._add_file_to_search_structure(newNode)
                node.childs[newNode.name] = newNode
            else:
                print("already exists",item.name)

            if item.is_dir():
                self._refresh(node.childs[item.name])
            else:
                # rehash needed calculate
                st = item.stat()
                _node = node.childs[item.name]
                if needs_rehash(_node, st):
                    self.changes_detector(_node)

        # delete part
        for name in internal:
            if name not in orignal_lst:
                self._delete_internal(node.childs[name],node)
                print("file deleted:",name)
    
    def refresh_cwd(self): # work good
        ''' start analysis and updating tree structure from cwd '''

        self._refresh(self.cwd)
        self.save_if_autosave()
    
    def refresh_root(self): # work good
        ''' start analysis and updating tree structure from root '''
        self._refresh(self.root)
        self.save_if_autosave()
    
    def refresh_node(self,node): # work good
        ''' start analysis and updating tree structure from given node '''
        self._refresh(node)
        self.save_if_autosave()
    
    def changes_detector(self,node):
    # implement lock feture in this later
        # for node in self.hash_queue():
        
        hash_ = file_hash(node.path)
        if node.hash != hash_:
            print('file content changes detected')
            
            node.hash = hash_ # update hash
            self.hash_Master.insert(node.path,hash_) # update and add hash in path_to_hash dictss
            node.state = "pending"
            result = name_ext(node.name)
            if result['ext'] in ALLOWED_TAGS_EXT:
                self.tag_queue.put(node)
            # genrate tags and assign  to them
            
        node.config_by_path(node.path)
    
    
    def search_hash_by_path(self, path: str):
        return self.hash_Master.search_hash_by_path(path)
        

    def search_paths_by_hash(self, file_hash: str):
        return self.hash_Master.search_paths_by_hash(file_hash)

    def search_duplicate_files(self):
        return self.hash_Master.search_duplicate_files()
        
 
        

    

    """
        ███████╗███████╗██╗     ███████╗ ██████╗████████╗
        ██╔════╝██╔════╝██║     ██╔════╝██╔════╝╚══██╔══╝
        ███████╗█████╗  ██║     █████╗  ██║        ██║   
        ╚════██║██╔══╝  ██║     ██╔══╝  ██║        ██║   
        ███████║███████╗███████╗███████╗╚██████╗   ██║   
        ╚══════╝╚══════╝╚══════╝╚══════╝ ╚═════╝   ╚═╝   
    """ 
    def select(self,node):  # work good
        if node not in self.selected_nodes:
            self.selected_nodes.append(node)
            print(f'select: {node}')
    
    def unselect(self,node): # work good
        if node in self.selected_nodes:
            self.selected_nodes.remove(node)
    
    def _select_all(self,node): # work good
        self.select(node)
        if node.type == 'dir':
            for name,child in node.childs.items():
                self._select_all(child)
    
    def select_all(self): # work good
        self._select_all(self.cwd)
    
    def unselect_all(self): # work good
        self.selected_nodes = []

    """
     ██████╗ ██████╗ ███████╗███╗   ██╗
    ██╔═══██╗██╔══██╗██╔════╝████╗  ██║
    ██║   ██║██████╔╝█████╗  ██╔██╗ ██║
    ██║   ██║██╔═══╝ ██╔══╝  ██║╚██╗██║
    ╚██████╔╝██║     ███████╗██║ ╚████║
     ╚═════╝ ╚═╝     ╚══════╝╚═╝  ╚═══╝
    """
    def _open_file_with_default_app(self,file_path):
        file_path = os.path.abspath(file_path)  # make path absolute
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return False

        try:
            if sys.platform == "win32":                    # Windows
                os.startfile(file_path)                    # simplest for Windows
            # elif sys.platform == "darwin":                 # macOS
            #     subprocess.run(["open", file_path])
            # else:                                          # Linux (Ubuntu, Fedora, etc.)
            #     subprocess.run(["xdg-open", file_path])
            print(f"Opened: {file_path}")
            return True
        except Exception as e:
            print(f"Could not open file: {e}")
            return False
        
    def open(self,node): # work good
        ''' open file with default application '''
        if node.type == 'file':
            self._open_file_with_default_app(node.path)
        else:
            if self.state == 'ideal':
                self.unselect_all()
            self.history.push(self.cwd)
            self.cwd = node
            print("changed directory to:",node.path)

    """
     ██╗███╗   ██╗███████╗ ██████╗ 
     ██║████╗  ██║██╔════╝██╔═══██╗
     ██║██╔██╗ ██║█████╗  ██║   ██║
     ██║██║╚██╗██║██╔══╝  ██║   ██║
     ██║██║ ╚████║██║     ╚██████╔╝
     ╚═╝╚═╝  ╚═══╝╚═╝      ╚═════╝
    """
    def info(self,node): # work good
        return node.to_dict()
    
    """
    ██████╗ ███████╗██╗     ███████╗████████╗███████╗
    ██╔══██╗██╔════╝██║     ██╔════╝╚══██╔══╝██╔════╝
    ██║  ██║█████╗  ██║     █████╗     ██║   █████╗  
    ██║  ██║██╔══╝  ██║     ██╔══╝     ██║   ██╔══╝  
    ██████╔╝███████╗███████╗███████╗   ██║   ███████╗
    ╚═════╝ ╚══════╝╚══════╝╚══════╝   ╚═╝   ╚══════╝
    """

    def _delete_internal(self,del_node,parent_node=None): # work good
        ''' delete file/folder from disk and tree structure '''
        if parent_node is None:
            parent_node = self.cwd
        if del_node.type == 'file':
            self.hash_Master.delete(del_node.path)
            parent_node._del_file_to_search_structure(del_node)

        elif del_node.type == 'dir':
            parent_node._del_dir_to_search_structure(del_node)
            for name,child in list(del_node.childs.items()):
                    self._delete_internal(child,del_node)
        del parent_node.childs[del_node.name]

    def _delete_memory(self,node): # work good
        ''' delete file/folder from memeory only '''
        if node.type == 'file':
            os.remove(node.path)
            print("file deleted from disk:",node.path)
        elif node.type == 'dir':
            shutil.rmtree(node.path)
            print("folder deleted from disk:",node.path)
    
    def delete(self,node): # work good
        ''' delete file/folder from disk and tree structure '''
        self._delete_memory(node)
        self._delete_internal(node)
        self.save_if_autosave()
        print("deleted:",node.path)
    
    """
     ██████╗██╗   ██╗████████╗     ██████╗ ██████╗ ██████╗ ██╗   ██╗    ██████╗  █████╗ ███████╗████████╗███████╗
    ██╔════╝██║   ██║╚══██╔══╝    ██╔════╝██╔═══██╗██╔══██╗╚██╗ ██╔╝    ██╔══██╗██╔══██╗██╔════╝╚══██╔══╝██╔════╝
    ██║     ██║   ██║   ██║       ██║     ██║   ██║██████╔╝ ╚████╔╝     ██████╔╝███████║███████╗   ██║   █████╗  
    ██║     ██║   ██║   ██║       ██║     ██║   ██║██╔═══╝   ╚██╔╝      ██╔═══╝ ██╔══██║╚════██║   ██║   ██╔══╝  
    ╚██████╗╚██████╔╝   ██║       ╚██████╗╚██████╔╝██║        ██║       ██║     ██║  ██║███████║   ██║   ███████╗
     ╚═════╝ ╚═════╝    ╚═╝        ╚═════╝ ╚═════╝ ╚═╝        ╚═╝       ╚═╝     ╚═╝  ╚═╝╚══════╝   ╚═╝   ╚══════╝
    """
    def _set_pointer(self,node): # work good
        self.pointer = node
        print("pointer set to:",node.path)

    def _get_pointer(self): # work good
        return self.pointer
    
    def cut(self): # work good
        if self.state == 'ideal':
            if self.selected_nodes:
                self.state = 'move'
                self._set_pointer(self.cwd)
                print("move mode activated")

    def copy(self): # work good
        if self.state == 'ideal':
            if self.selected_nodes:
                self.state = 'copy'
                self._set_pointer(self.cwd)
                print("copy mode activated")
    
    def _paste_for_move(self): # work good
            if self.selected_nodes:
                for node in self.selected_nodes:
                    shutil.move(node.path,self.cwd.path)
                    print(f"moved {node.path} to {self.cwd.path}")
                    self._delete_internal(node,self._get_pointer())
                print("paste completed, back to ideal mode")

    def _paste_for_copy(self): # work good
            if self.selected_nodes:
                for node in self.selected_nodes:
                    if node.type == 'file':
                        shutil.copy2(node.path,self.cwd.path)
                        print(f"copied {node.path} to {self.cwd.path}")
                    elif node.type == 'dir':
                        dest_path = os.path.join(self.cwd.path,node.name)
                        shutil.copytree(node.path,dest_path)
                        print(f"copied directory {node.path} to {dest_path}")
                print("paste completed, back to ideal mode")

    def paste(self): # work good
        '''there is a issue of already exist thing'''#<<<<<<<<<<ERROR
        if self.state == 'move':
            self._paste_for_move()
        elif self.state == 'copy':
            self._paste_for_copy()
        self.unselect_all()
        self.refresh_cwd()
        self.state = 'ideal'
    
    """
     ██████╗██████╗ ███████╗ █████╗ ████████╗███████╗
    ██╔════╝██╔══██╗██╔════╝██╔══██╗╚══██╔══╝██╔════╝
    ██║     ██████╔╝█████╗  ███████║   ██║   █████╗  
    ██║     ██╔══██╗██╔══╝  ██╔══██║   ██║   ██╔══╝  
    ╚██████╗██║  ██║███████╗██║  ██║   ██║   ███████╗
     ╚═════╝╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝   ╚═╝   ╚══════╝
    """
    def _write_content_to_file(self,filepath,content):
        with open(filepath,"w") as f:
            f.write(f'{content}\n')

    def _append_content_to_file(self,filepath,content):
        with open(filepath,"a") as f:
            f.write(f'{content}\n')

    def _config_node(self,node):
        stat = os.stat(node.path)
        node._config_stat(stat)

            
    def _create_dir_memory(self,dir_name,p_node= None): # work good
        if p_node is None:
            p_node = self.cwd
        new_dir_path = os.path.join(p_node.path,dir_name)
        os.makedirs(new_dir_path,exist_ok=True)
        print("directory created at:",new_dir_path) 
    
    def _create_dir_internal(self,dir_name,p_node= None): # work good
        if p_node is None:
            p_node = self.cwd
        newNode = Node(dir_name,self.normalize_path(os.path.join(p_node.path,dir_name)))
        newNode.type = 'dir'
        p_node.childs[dir_name] = newNode
        p_node._add_dir_to_search_structure(newNode)
        print("directory node created at:",newNode.path)
        return newNode
    
    def _create_file_memory(self,file_name,content="",p_node= None): # work good
        if p_node is None:
            p_node = self.cwd
        new_file_path = os.path.join(p_node.path,file_name)
        self._write_content_to_file(new_file_path,content)
        print("file created at:",new_file_path)
    
    def _create_file_internal(self,file_name,p_node= None): # work good
        if p_node is None:
            p_node = self.cwd
        newNode = Node(file_name,self.normalize_path(os.path.join(p_node.path,file_name)))
        newNode.type = 'file'
        p_node.childs[file_name] = newNode
        p_node._add_file_to_search_structure(newNode)
        print("file node created at:",newNode.path)
        return newNode

    def create_dir(self,dir_name,p_node= None):
        self._create_dir_memory(dir_name,p_node)
        node = self._create_dir_internal(dir_name,p_node)
        self._config_node(node)
        self.save_if_autosave()

    def create_file(self,file_name,content="",p_node= None):
        self._create_file_memory(file_name,content,p_node)
        node = self._create_file_internal(file_name,p_node)
        self.hash_queue.append(node)
        self._config_node(node)
        self.save_if_autosave()
    
    def write_to_file(self,node,content):
        if node.type == 'file':
            self._write_content_to_file(node.path,content)
            self._config_node(node)
            self.save_if_autosave()
            print("written to file:",node.path)
        else:
            print("cannot write to a directory")
    
    def append_to_file(self,node,content):
        if node.type == 'file':
            self._append_content_to_file(node.path,content)
            self._config_node(node)
            self.save_if_autosave()
            print("appended to file:",node.path)
        else:
            print("cannot append to a directory")

    """
     █████╗ ██████╗ ██████╗ ██╗████████╗██╗ ██████╗ ███╗   ██╗ █████╗ ██╗     
    ██╔══██╗██╔══██╗██╔══██╗██║╚══██╔══╝██║██╔═══██╗████╗  ██║██╔══██╗██║     
    ███████║██║  ██║██║  ██║██║   ██║   ██║██║   ██║██╔██╗ ██║███████║██║     
    ██╔══██║██║  ██║██║  ██║██║   ██║   ██║██║   ██║██║╚██╗██║██╔══██║██║     
    ██║  ██║██████╔╝██████╔╝██║   ██║   ██║╚██████╔╝██║ ╚████║██║  ██║███████╗
    ╚═╝  ╚═╝╚═════╝ ╚═════╝ ╚═╝   ╚═╝   ╚═╝ ╚═════╝ ╚═╝  ╚═══╝╚═╝  ╚═╝╚══════╝
    """

    def show_list(self,filter = None): # work good
        ''' show list of files and folders in current working directory '''
        print(f"Contents of directory: {self.cwd.path}")
        result = []
        for name,child in self.cwd.childs.items():
            result.append(child.to_dict())
        return result

    def go_to_root(self): # work good
        self.cwd = self.root
        print("changed directory to root:",self.root.path)
    
    def go_to(self,name): # work good
        if self.state == 'ideal':
            self.unselect_all()
        if name in self.cwd.childs:
            node = self.cwd.childs[name]
            self.open(node)
        else:
            print(f"{name} not found in current directory")
    
    def go_back(self): # work good
        if self.state == 'ideal':
            self.unselect_all()
        if not self.history.container:
            print("No previous directory in history")
            return
        prev_dir = self.history.pop()
        self.cwd = prev_dir
        print("changed directory to:",self.cwd.path)

    def get_node(self,filename): # work good
        if filename in self.cwd.childs.keys():
            return self.cwd.childs[filename]
        else:
            return None

    def path_verifier(self,path):
        return True if self.root.path in path else False

    def path_breaker(self,path):
        rootv=self.root.path.split('/')
        pathv=path.split('/')
        l = len(rootv)
        return pathv[4:]
    
    def get_node_by_path(self,path):
        if self.path_verifier(path):
            results = self.path_breaker(path)
            root = self.root
            for pathname in results:
                if pathname in root.childs:
                    root = root.childs[pathname]
                elif pathname:
                    return {
                        'result':None,
                        'message':'file not exist'
                    }
              
            return {
                    'result':root,
                    'message':'file exist'
            }
        return {
                    'result':None,
                    'message':'path is incorrect or formate is incorrect'
            }



        
    """
    ██████╗ ███████╗███╗   ██╗ █████╗ ███╗  ╔██╗███████╗
    ██╔══██╗██╔════╝████╗  ██║██╔══██╗████ ╔███║██╔════╝
    ██████╔╝█████╗  ██╔██╗ ██║███████║██╔███═██║█████╗  
    ██╔══██╗██╔══╝  ██║╚██╗██║██╔══██║██║    ██║██╔══╝  
    ██║  ██║███████╗██║ ╚████║██║  ██║██║    ██║███████╗
    ╚═╝  ╚═╝╚══════╝╚═╝  ╚═══╝╚═╝  ╚═╝╚═╝    ╚═╝╚══════╝
    """
    def _rename_internal(self,filename,node,p_node=None):
        if p_node is None:
            p_node = self.cwd
        if node.type == 'dir':
            p_node._del_dir_to_search_structure(node)    
        else:
            self.hash_Master.delete(node.path)
            p_node._del_file_to_search_structure(node)    

        node.path  = node.path[:-len(node.name)]+filename #path change
        

        del p_node.childs[node.name]
        p_node.childs[filename] = node
        
        node.name = filename
        self._config_node(node)
        if node.type == 'dir':
            p_node._add_dir_to_search_structure(node)    
        else:
            p_node._add_file_to_search_structure(node)
            self.hash_queue.append(node)    
     
       
    def _rename_memory(self,name,node):
        old_add = node.path
        new_add = node.path[:-len(node.name)]+name
        os.rename(old_add,new_add)

    def rename(self,name,node,p_node=None):
        self._rename_memory(name,node)
        self._rename_internal(name,node,p_node)
    """
            ███████╗███████╗ █████╗ ██████╗  ██████╗██╗  ██╗
            ██╔════╝██╔════╝██╔══██╗██╔══██╗██╔════╝██║  ██║
            ███████╗█████╗  ███████║██████╔╝██║     ███████║
            ╚════██║██╔══╝  ██╔══██║██╔══██╗██║     ██╔══██║
            ███████║███████╗██║  ██║██║  ██║╚██████╗██║  ██║
            ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝
    """ 
    def search_helper_prifix(self,node,prifix,results):
        result = node.search_prifix(prifix)
        results += result
        for item in node.childs.values():
            if item.type == 'dir':
                self.search_helper_prifix(item,prifix,results)
        return results

    def search_helper_ext(self,node,ext,results):
        result = node.search_ext(ext)
        results += result
        for item in node.childs.values():
            if item.type == 'dir':
                self.search_helper_ext(item,ext,results)
        return results

    def search_prifix(self,prifix):
        results = []
        self.search_helper_prifix(self.cwd,prifix,results)
        return results

    def search_prifix_all(self,prifix):
        results = []
        self.search_helper_prifix(self.root,prifix,results)
        return results
    
    def search_ext(self,ext):
        results = []
        self.search_helper_ext(self.cwd,ext,results)
        return results

    def search_ext_all(self,ext):
        results = []
        self.search_helper_ext(self.root,ext,results)
        return results


    def update_config_based_on_time_and_give_tags(self):
        pass
            

In [28]:
fs = FSManager([])


In [29]:
fs.load()

In [30]:
fs.refresh_root()

already exists hkdjbkjsa
already exists z.txt
new file or folder detected: zzzz
Inserted: zzzz -> d:/don'tDelete/Desktop/file management system/testnew/hkdjbkjsa/zzzz
already exists xyzksjkdnjkdjn.txt
new file or folder detected: xyzksjkdnjkdjn.txtuditya
Inserted: xyzksjkdnjkdjn.txtuditya -> d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txtuditya
Inserted 'xyzksjkdnjkdjn.txtuditya' with path 'd:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txtuditya' under extension 'txtuditya'
file content changes detected


In [31]:
fs.cwd.childs

{'hkdjbkjsa': Node(name=hkdjbkjsa, path=d:/don'tDelete/Desktop/file management system/testnew/hkdjbkjsa, type=dir),
 'xyzksjkdnjkdjn.txt': Node(name=xyzksjkdnjkdjn.txt, path=d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txt, type=file),
 'xyzksjkdnjkdjn.txtuditya': Node(name=xyzksjkdnjkdjn.txtuditya, path=d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txtuditya, type=file)}

In [32]:
fs.show_list()

Contents of directory: d:/don'tDelete/Desktop/file management system/testnew


[{'name': 'hkdjbkjsa',
  'path': "d:/don'tDelete/Desktop/file management system/testnew/hkdjbkjsa",
  'type': 'dir',
  'size': 0,
  'modified_time': 1768650831.7428799,
  'created_time': 1768387734.8145292,
  'mode': 16895,
  'childs': ['z.txt', 'zzzz']},
 {'name': 'xyzksjkdnjkdjn.txt',
  'path': "d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txt",
  'type': 'file',
  'size': 6,
  'modified_time': 1768389001.8071804,
  'created_time': 1768389001.8054922,
  'mode': 33206,
  'childs': ['xyzksjkdnjkdjn.txtuditya']},
 {'name': 'xyzksjkdnjkdjn.txtuditya',
  'path': "d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txtuditya",
  'type': 'file',
  'size': 0,
  'modified_time': 1768388973.1610885,
  'created_time': 1768388973.1610885,
  'mode': 33206,
  'childs': []}]

In [42]:
class CLI:
    def __init__(self,fs) -> None:
        self.fs = fs
        self.isRunning = True
        self.last_input = None

    def lst(self):
        print(self.fs.root.childs)

In [43]:
cli = CLI(fs)

In [44]:
cli.lst()

{'hkdjbkjsa': Node(name=hkdjbkjsa, path=d:/don'tDelete/Desktop/file management system/testnew/hkdjbkjsa, type=dir), 'xyzksjkdnjkdjn.txt': Node(name=xyzksjkdnjkdjn.txt, path=d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txt, type=file), 'xyzksjkdnjkdjn.txtuditya': Node(name=xyzksjkdnjkdjn.txtuditya, path=d:/don'tDelete/Desktop/file management system/testnew/xyzksjkdnjkdjn.txtuditya, type=file)}


In [1]:
with open("w/w/w/w/w/n.txt", "wb") as f:
    pickle.dump("data", f, protocol=pickle.HIGHEST_PROTOCOL)

FileNotFoundError: [Errno 2] No such file or directory: 'w/w/w/w/w/n.txt'

In [ ]:
class LogHandler:
    def __init__(self,path) -> None:
        self.path = path


    def append(self,message):
        with open(self.path,"a") as f:
            f.write(message)


TypeError: 'list' object does not support the context manager protocol

In [1]:
class Stack:
    def __init__(self):
        self.container = []
    
    def push(self,value):
        self.container.append(value)
        return value
    
    def pop(self):
        return self.container.pop(-1)
    
    def top(self):
        return self.container[-1]

    def empty(self):
        return True if not self.container else False

In [4]:
st = Stack()
st.push(1)

1

In [5]:
st.empty()

False

In [2]:
from FS.constant import DEFAULT_NAME,DEFAULT_PATH,ALLOWED_TAGS_EXT
from FS.stack import Stack
from FS.prifixSearch import Trie
from FS.hash import HashMaster
from FS.extSearch import CustomHashMap
from FS.fileReader import universal_reader
from FS.useful import needs_rehash,file_hash,name_ext
from FS.cusq import Que

# import subprocess
# from mrvector import MrVectorExpert
import os
import sys
import shutil
import hashlib
from pathlib import Path
import pickle
from queue import Queue
import time
import logging

logger = logging.getLogger("FS")


class Node:
    def __init__(self,name,path):
        self.path = path
        self.name = name
        self.childs = {}
        self.type = 'dir'

        self.state = "indexed"

        self.prifixSearch = Trie()
        self.extensionSearch =  CustomHashMap()

        self.hash = None
        self.vector = None
        self.tags = None

        # st.st_size
        self.size = None
        # st.st_mtime
        self.modified_time = None
        # st.st_ctime
        self.created_time = None
        # st.st_mode
        self.mode = None

    def _config_stat(self,st):

        self.size = st.st_size
        self.modified_time = st.st_mtime
        self.created_time = st.st_ctime
        self.mode = st.st_mode
    
    def config(self,item):
        logger.info(f"config > {self.name}")
        st = item.stat()
        self._config_stat(st)
        if item.is_dir():
            self.type = 'dir'
        else:
            self.type = 'file'
    def config_by_path(self,path):
        st = os.stat(path)
        self._config_stat(st)

    def _add_file_to_search_structure(self,node):
        self.prifixSearch.insert(node.name,node.path)
        self.extensionSearch.insert(node.name,node.path)
    
    def _del_file_to_search_structure(self,node):
        self.prifixSearch.delete(node.name,node.path)
        self.extensionSearch.delete(node.name,node.path)

    def _add_dir_to_search_structure(self,node):
        self.prifixSearch.insert(node.name,node.path)

    def _del_dir_to_search_structure(self,node):
        self.prifixSearch.delete(node.name,node.path)

    def search_prifix(self,prifix):
        return self.prifixSearch.search_by_prefix(prifix)
    
    def search_ext(self,ext):
        return self.extensionSearch.get_by_extension(ext)

    def to_dict(self):
        return {
            'name': self.name,
            'path': self.path,
            'type': self.type,
            'size': self.size,
            'modified_time': self.modified_time,
            'created_time': self.created_time,
            'mode': self.mode,
            'childs': [name for name in self.childs.keys()]
        }
    
    
    def __repr__(self):
        return f"Node(name={self.name}, path={self.path}, type={self.type})"




class FSManager:
    '''file management'''
    def __init__(self,tagG,mrvec,db_path="save/root.pkl", autosave=True):
        self.default = DEFAULT_PATH
        self.default_name = DEFAULT_NAME
        self.tagG = tagG
        self.root = None
        self.root_store_path = db_path
        self.autosave = autosave
        # self.prifixSearch = Trie()
        # self.extensionSearch = CustomHashMap()
        self.cwd = self.root
        self.selected_nodes = []
        self.state = 'ideal'
        self.pointer = None
        self.history = Stack()
        self.hash_queue = []
        self.hash_Master = HashMaster()
        self.mrvec = mrvec
        self.tag_queue = Que()
        self.isRootRefresh = False

    def normalize_path(self,path):
        return Path(path).as_posix()

    def save(self):
        """
        Save HashMaster state to pickle
        """
        data = {
            "root": self.root
        }
        with open(self.root_store_path, "wb") as f:
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    def save_if_autosave(self):
        if self.autosave:
            self.save()

    def load(self):
        """
        Load HashMaster state from pickle
        """
        if not os.path.exists(self.root_store_path):
            logger.error("root.pkl file not exist!")
            
            self.root = Node(self.default_name,self.default)
            self.cwd = self.root
            self.refresh_cwd()
            
            return

        try:
            with open(self.root_store_path, "rb") as f:
                data = pickle.load(f)

            self.root = data.get("root", Node(self.default_name,self.default))
            if self.root.name != self.default_name:
                raise Exception("root change")
            self.cwd = self.root
            logger.info(f"{self.root_store_path} is loaded succesfully")

        except Exception:
            # Corrupted pickle → safe reset
            logger.warning(f"{self.root_store_path} Corrupted pickle → safe reset")
            self.root = Node(self.default_name,self.default)
            self.cwd = self.root
        self.refresh_cwd()

    '''                                                                                
        ██████╗ ███████╗███████╗██████╗ ███████╗███████╗██╗  ██╗
        ██╔══██╗██╔════╝██╔════╝██╔══██╗██╔════╝██╔════╝██║  ██║
        ██████╔╝█████╗  █████╗  ██████╔╝█████╗  ███████╗███████║
        ██╔══██╗██╔══╝  ██╔══╝  ██╔══██╗██╔══╝  ╚════██║██╔══██║
        ██║  ██║███████╗██║     ██║  ██║███████╗███████║██║  ██║
        ╚═╝  ╚═╝╚══════╝╚═╝     ╚═╝  ╚═╝╚══════╝╚══════╝╚═╝  ╚═╝    
    '''
    def _refresh(self, root_node):
        stack = [root_node]

        while stack:
            node = stack.pop()

            try:
                internal = list(node.childs.keys())
                original = list(os.scandir(node.path))
                original_names = [f.name for f in original]

                # create + update
                for item in original:
                    if item.name not in internal:
                        print(f"new file/dir detected > {item.name}")
                        newNode = Node(item.name, self.normalize_path(item.path))
                        newNode.config(item)

                        if item.is_dir():
                            node._add_dir_to_search_structure(newNode)
                        else:
                            node._add_file_to_search_structure(newNode)

                        node.childs[newNode.name] = newNode
                    else:

                        print(f'file areday exist {item.name}')

                    if item.is_dir():
                        stack.append(node.childs[item.name])
                    else:
                        st = item.stat()
                        node_ = node.childs[item.name]
                        if needs_rehash(node_, st):
                            self.changes_detector(node_)

                    yield   # 🔥 MICRO STEP

                # delete
                for name in internal:
                    if name not in original_names:
                        self._delete_internal(node.childs[name], node)

                    yield   # 🔥 MICRO STEP

            except Exception as e:
                print(f'{e}')
                yield

    # def _refresh(self,node): # work good
    #     #>>>>>>>>>>>>>>>>>>WE NEED TO IMPLEMENT THE MODIFICATION TIME DETECTION AND AFTER THAT CONFIG NODE 
    #     #  CHECK FULL CODE AND ANALYIS WHERE THE CONFIG NODE IS NEEDED SO THE TREE BECOME UPTO DATE
    #     #  THINK ABOUT WE NEED HASH OR NOT FOR CHANGE DETECTION 
    #     # IMPLEMET PICKEL PART TO STORE HASHMAP GRAPH TIER 
    #     internal = list(node.childs.keys())
    #     orignal = list(os.scandir(node.path))
    #     orignal_lst = [f.name for f in orignal]
    #     #  create part
    #     for item in orignal:
    #         if item.name not in internal:
    #             # new file or folder added
    #             logger.info("new file or folder detected:",item.name)
    #             newNode = Node(item.name,self.normalize_path(item.path))
    #             newNode.config(item)
    #             if item.is_dir():
    #                 node._add_dir_to_search_structure(newNode)
    #             else:
    #                 node._add_file_to_search_structure(newNode)
    #             node.childs[newNode.name] = newNode
    #         else:
    #             logger.info("already exists",item.name)

    #         if item.is_dir():
    #             self._refresh(node.childs[item.name])
    #         else:
    #             # rehash needed calculate
    #             st = item.stat()
    #             node_ = node.childs[item.name]
    #             if needs_rehash(node_, st):
    #                 self.changes_detector(node_)

    #     # delete part
    #     for name in internal:
    #         if name not in orignal_lst:
    #             self._delete_internal(node.childs[name],node)
    #             logger.info("file deleted:",name)
    
    def refresh_cwd(self): # work good
        ''' start analysis and updating tree structure from cwd '''
        print(f"refresh start cwd > {self.cwd.path}")
        for _ in self._refresh(self.cwd):
            pass
        print(f"refresh complete cwd > {self.cwd.path}")
        self.save_if_autosave()
    
    def refresh_root(self): # work good
        ''' start analysis and updating tree structure from root '''
        print("full root loading started")
        for _ in self._refresh(self.root):
            pass
        print("full root loading completed")
        self.save_if_autosave()

    
    def refresh_node(self,node): # work good
        ''' start analysis and updating tree structure from given node '''
        print(f"refresh start > {node.path}")
        for _ in self._refresh(node):
            pass
        print(f"refresh complete > {node.path}")
        self.save_if_autosave()
    
    def changes_detector(self,node):

        hash_ = file_hash(node.path)
        if node.hash != hash_:
            print(f'file content changes detected > {node.path}')
            
            node.hash = hash_ # update hash
            self.hash_Master.insert(node.path,hash_) # update and add hash in path_to_hash dictss
            result = name_ext(node.name)
            if result['ext'] in ALLOWED_TAGS_EXT:
                node.state = "pending"
                self.tag_queue.put(node.path)
                print(f"{node.name} > tag queue")
            # genrate tags and assign  to them
            
        node.config_by_path(node.path)
    
  
    
    

    """
        ███████╗███████╗██╗     ███████╗ ██████╗████████╗
        ██╔════╝██╔════╝██║     ██╔════╝██╔════╝╚══██╔══╝
        ███████╗█████╗  ██║     █████╗  ██║        ██║   
        ╚════██║██╔══╝  ██║     ██╔══╝  ██║        ██║   
        ███████║███████╗███████╗███████╗╚██████╗   ██║   
        ╚══════╝╚══════╝╚══════╝╚══════╝ ╚═════╝   ╚═╝   
    """ 
    def select(self,node):  # work good
        if node not in self.selected_nodes:
            self.selected_nodes.append(node)
            logger.info(f'select > {node.name}')
    
    def unselect(self,node): # work good
        if node in self.selected_nodes:
            self.selected_nodes.remove(node)
            logger.info(f'unselect > {node.name}')
    
    def _select_all(self,node): # work good
        self.select(node)
        if node.type == 'dir':
            for name,child in node.childs.items():
                self._select_all(child)
    
    def select_all(self): # work good
        self._select_all(self.cwd)
    
    def unselect_all(self): # work good
        self.selected_nodes = []
        logger.info('unselect all')

    """
     ██████╗ ██████╗ ███████╗███╗   ██╗
    ██╔═══██╗██╔══██╗██╔════╝████╗  ██║
    ██║   ██║██████╔╝█████╗  ██╔██╗ ██║
    ██║   ██║██╔═══╝ ██╔══╝  ██║╚██╗██║
    ╚██████╔╝██║     ███████╗██║ ╚████║
     ╚═════╝ ╚═╝     ╚══════╝╚═╝  ╚═══╝
    """
    def _open_file_with_default_app(self,file_path):
        file_path = os.path.abspath(file_path)  # make path absolute
        if not os.path.exists(file_path):
            logger.error(f"open > {file_path} : not found")
            return False

        try:
            if sys.platform == "win32":                    # Windows
                os.startfile(file_path)                    # simplest for Windows

            logger.info(f"open > {file_path} : successfully")

            return True
        except Exception as e:
            logger.info(f" {e}")
            logger.error(f"open > {file_path} :Could not open file > {e}")
            return False
    
    def set_cwd(self,node):
        self.history.push(node)
        self.cwd = node
        self.refresh_cwd()
        logger.info(f"changed cwd > {node.path}")


    def open(self,node): # work good
        ''' open file with default application '''
        if node.type == 'file':
            self._open_file_with_default_app(node.path)
        else:
            if self.state == 'ideal':
                self.unselect_all()
            self.set_cwd(node)
            logger.info(f"changed cwd >{node.path}")

    
    """
    ██████╗ ███████╗██╗     ███████╗████████╗███████╗
    ██╔══██╗██╔════╝██║     ██╔════╝╚══██╔══╝██╔════╝
    ██║  ██║█████╗  ██║     █████╗     ██║   █████╗  
    ██║  ██║██╔══╝  ██║     ██╔══╝     ██║   ██╔══╝  
    ██████╔╝███████╗███████╗███████╗   ██║   ███████╗
    ╚═════╝ ╚══════╝╚══════╝╚══════╝   ╚═╝   ╚══════╝
    """

    def _delete_internal(self,del_node,parent_node=None): # work good
        ''' delete file/folder from disk and tree structure '''
        if parent_node is None:
            parent_node = self.cwd
        if del_node.type == 'file':
            self.hash_Master.delete(del_node.path)
            self.mrvec.delete(del_node.path)
            parent_node._del_file_to_search_structure(del_node)

        elif del_node.type == 'dir':
            parent_node._del_dir_to_search_structure(del_node)
            for name,child in list(del_node.childs.items()):
                    self._delete_internal(child,del_node)
        del parent_node.childs[del_node.name]
        logger.info(f"delete-internal>{del_node.type} > {del_node.path}")

    def _delete_memory(self,node): # work good
        ''' delete file/folder from memeory only '''
        if node.type == 'file':
            os.remove(node.path)
            logger.info(f"delete-memory > file > {node.path}")
        elif node.type == 'dir':
            shutil.rmtree(node.path)
            logger.info(f"delete-memory > dir > {node.path}")
    
    def delete(self,node): # work good
        ''' delete file/folder from disk and tree structure '''
        self._delete_memory(node)
        self._delete_internal(node)
        self.save_if_autosave()
    

    

    """
     █████╗ ██████╗ ██████╗ ██╗████████╗██╗ ██████╗ ███╗   ██╗ █████╗ ██╗     
    ██╔══██╗██╔══██╗██╔══██╗██║╚══██╔══╝██║██╔═══██╗████╗  ██║██╔══██╗██║     
    ███████║██║  ██║██║  ██║██║   ██║   ██║██║   ██║██╔██╗ ██║███████║██║     
    ██╔══██║██║  ██║██║  ██║██║   ██║   ██║██║   ██║██║╚██╗██║██╔══██║██║     
    ██║  ██║██████╔╝██████╔╝██║   ██║   ██║╚██████╔╝██║ ╚████║██║  ██║███████╗
    ╚═╝  ╚═╝╚═════╝ ╚═════╝ ╚═╝   ╚═╝   ╚═╝ ╚═════╝ ╚═╝  ╚═══╝╚═╝  ╚═╝╚══════╝
    """

    def show_list(self,filter = None): # work good
        ''' show list of files and folders in current working directory '''
        print(f"call show list>: {self.cwd.path}")
        result = []
        for name,child in self.cwd.childs.items():
            result.append(child.to_dict())
        return result

    

    def go_to_root(self): # work good
        self.set_cwd(self.root)
        
    
    def go_to(self,name): # work good
        if self.state == 'ideal':
            self.unselect_all()
        if name in self.cwd.childs:
            node = self.cwd.childs[name]
            self.set_cwd(node)
        else:
            logger.info(f"{name} not found in cwd")
    
    def go_back(self): # work good
        if self.state == 'ideal':
            self.unselect_all()
        if self.history.empty():
            logger.info("history stack is Empty")
            return
        prev_node = self.history.pop()
        self.set_cwd(prev_node)

    def get_node(self,filename): # work good
        if filename in self.cwd.childs.keys():
            return self.cwd.childs[filename]
        else:
            return None

    def path_verifier(self,path):
        return True if self.root.path in path else False

    def path_breaker(self,path):
        rootv=self.root.path.split('/')
        pathv=path.split('/')
        l = len(rootv)
        return pathv[4:]
    
    def get_node_by_path(self,path):
        if self.path_verifier(path):
            results = self.path_breaker(path)
            root = self.root
            for pathname in results:
                if pathname in root.childs:
                    root = root.childs[pathname]
                elif pathname:
                    return {
                        'result':None,
                        'message':'file not exist'
                    }
              
            return {
                    'result':root,
                    'message':'file exist'
            }
        return {
                    'result':None,
                    'message':'path is incorrect or formate is incorrect'
            }



       

In [70]:
fs = FSManager([],[])

In [71]:
fs.load()

refresh start cwd > d:/don'tDelete/Desktop/file management system/testnew
file areday exist fruit.txt
file areday exist jj
file areday exist udii
file areday exist xyzksjkdnjkdjn.txt
file areday exist z.txt
file areday exist zzzz
refresh complete cwd > d:/don'tDelete/Desktop/file management system/testnew


In [72]:
fs.refresh_root()

full root loading started
file areday exist fruit.txt
file areday exist jj
file areday exist udii
file areday exist xyzksjkdnjkdjn.txt
file areday exist z.txt
file areday exist zzzz
full root loading completed


In [73]:
fs.root.childs

{'fruit.txt': Node(name=fruit.txt, path=d:/don'tDelete/Desktop/file management system/testnew/fruit.txt, type=file),
 'jj': Node(name=jj, path=d:/don'tDelete/Desktop/file management system/testnew/jj, type=dir),
 'udii': Node(name=udii, path=d:/don'tDelete/Desktop/file management system/testnew/udii, type=dir)}

In [11]:
a = " cd  file management system"

In [13]:
c,name = a.strip().split(" ",1)
c,name

('cd', ' file management system')

In [ ]:
class CLI:
    def __init__(self,fs) -> None:
        self.fs = fs
        self.isRunning = True
        self.last_input = None
    
    def error(self):
        return "command is not founds as good"

    def _ls(self,command):
        base = command.split()
        if len(base) == 1:
            return list(self.cwd.childs.keys())
        else:
            self.error()

    def _cd(self,command):
        c,name = a.strip().split(" ",1)

        pass
    

    def _back(self,command):
        pass
    

    def _root(self,command):
        pass
    

    def _refresh(self,command):
        pass
    

    def _select(self,command):
        pass
    

    def _unselect(self,command):
        pass
    

    def _open(self,command):
        pass
    

    def _cut(self,command):
        pass
    

    def _copy(self,command):
        pass
    

    def _del(self,command):
        pass
    

    def _mkdir(self,command):
        pass
    

    def _mkf(self,command):
        pass
    

    def _write(self,command):
        pass
    

    def _append(self,command):
        pass
    

    def _info(self,command):
        pass
    

    def _rename(self,command):
        pass
    

    def _tree(self,command):
        pass
    

    def _search(self,command):
        pass
    

    def _find(self,command):
        pass
    

    def _exit(self,command):
        pass
    

    def _pwd(self,command):
        pass
    

    def _tag(self,command):
        pass
    

    def _verify(self,command):
        pass
    

    def _status(self,command):
        pass
    

    def _stats(self,command):
        pass
    


    

    def process_user_task(self,command):
        base = command.split()[0]
        
        commands = {
            "ls":self._ls,
            "cd":self._cd,
            "back":self._back,
            "root":self._root,
            "refresh":self._refresh,
            "select":self._select,
            "unselect":self._unselect,
            "open":self._open,
            "cut":self._cut,
            "copy":self._copy,
            "del":self._del,
            "mkdir":self._mkdir,
            "mkf":self._mkf,
            "write":self._write,
            "append":self._append,
            "info":self._info,
            "rename":self._rename,
            "tree":self._tree,
            "search":self._search,
            "find":self._find,
            "exit":self._exit,
            "pwd":self._pwd,
            "tag":self._tag,
            "verify":self._verify,
            "status":self._status,
            "stats":self._stats
        }
        commands[base]()



        if command == 'quit':
            self.isRunning = False
        elif command.startswith('cd '):
            _,name = command.split(' ',1)
            self.fs.go_to(name)
        elif command == 'back':
            self.fs.go_back()
        elif command == 'root':
            self.fs.go_to_root()
        elif command == 'refresh':
            self.fs.refresh_cwd()
        elif command == 'refreshall':
            self.fs.refresh_root()
        elif command == 'selectall':
            self.fs.select_all()
        elif command.startswith('select'):
            _,name = command.split(' ',1)
            node = self.fs.get_node(name)
            if node:
                self.fs.select(node)
            else:
                print(f"{name} not found in current directory")
        elif command == "unselectall":
            self.fs.unselect_all()
        elif command.startswith('unselect'):
            _,name = command.split(' ',1)
            node = self.fs.get_node(name)
            if node:
                self.fs.unselect(node)
            else:
                print(f"{name} not found in current directory")
        elif command.startswith('open'):
            _,name = command.split(' ',1)
            node = self.fs.get_node(name)
            if node:
                self.fs.open(node)
            else:
                print(f"{name} not found in current directory")
        elif command == 'cut':
            self.fs.cut()
        elif command == 'copy':
            self.fs.copy()
        elif command == 'paste':
            self.fs.paste()
        elif command.startswith('del'):
            _,name = command.split(' ',1)
            node = self.fs.get_node(name)
            if node:
                self.fs.delete(node)
            else:
                print(f"{name} not found in current directory")
        elif command.startswith('mkdir'):
            _,name = command.split(' ',1)
            self.fs.create_dir(name)
        elif command.startswith('mkf'):
            _,name,content = command.split(' ',2)
            self.fs.create_file(name,content) 
        elif command.startswith('rename'):
            _,old_name,new_name = command.split(' ',2)
            node = self.fs.get_node(old_name)
            if node:
                self.fs.rename(new_name,node)
            else:
                print(f"{old_name} not found in current directory")
        elif command.startswith('write'):
            _,name,content = command.split(' ',2)
            node = self.fs.get_node(name)
            if node:
                self.fs.write_to_file(node,content)
            else:
                print(f"{name} not found in current directory")
        elif command.startswith('append'):
            _,name,content = command.split(' ',2)
            node = self.fs.get_node(name)
            if node:
                self.fs.append_to_file(node,content)
            else:
                print(f"{name} not found in current directory")
        elif command.startswith('search'):
            _,typ,mood,content = command.split(' ',3)
            res = []
            if typ == '-p':
                if mood == '-n':
                    res = self.fs.search_prifix(content)
                elif mood == '-a':
                    res = self.fs.search_prifix_all(content)
                else:
                    print("mood is invalid")
            elif typ == '-e':
                if mood == '-n':
                    res = self.fs.search_ext(content)
                elif mood == '-a':
                    res = self.fs.search_ext_all(content)
                else:
                    print("mood is invalid")
            else:
                    print("type is invalid")
   

In [34]:
process_user_task(a)

hlo


In [35]:
lst = [
    "ls",
    "cd",
    "back",
    "root",
    "refresh",
    "select",
    "unselect",
    "open",
    "cut",
    "copy",
    "del",
    "mkdir",
    "mkf",
    "write",
    "append",
    "info",
    "rename",
    "tree",
    "search",
    "find",
    "exit",
    "pwd",
    "tag",
    "verify",
    "status",
    "stats"
]

In [40]:
for i in lst:
    print(f'''
    def _{i}(self,command):
        pass
    ''')


    def _ls(self,command):
        pass
    

    def _cd(self,command):
        pass
    

    def _back(self,command):
        pass
    

    def _root(self,command):
        pass
    

    def _refresh(self,command):
        pass
    

    def _select(self,command):
        pass
    

    def _unselect(self,command):
        pass
    

    def _open(self,command):
        pass
    

    def _cut(self,command):
        pass
    

    def _copy(self,command):
        pass
    

    def _del(self,command):
        pass
    

    def _mkdir(self,command):
        pass
    

    def _mkf(self,command):
        pass
    

    def _write(self,command):
        pass
    

    def _append(self,command):
        pass
    

    def _info(self,command):
        pass
    

    def _rename(self,command):
        pass
    

    def _tree(self,command):
        pass
    

    def _search(self,command):
        pass
    

    def _find(self,command):
        pass
    

    def _exit(self,command):


In [46]:
a = "rename -f old name -b new name    "

In [1]:
a = {'name': 'fruit.txt', 'path': "d:/don'tDelete/Desktop/file management system/testnew/fruit.txt", 'type': 'file', 'size': 36, 'modified_time': 1769247605.8295078, 'created_time': 1769247461.8377604, 'mode': 33206}

In [3]:
list(a.values())

['fruit.txt',
 "d:/don'tDelete/Desktop/file management system/testnew/fruit.txt",
 'file',
 36,
 1769247605.8295078,
 1769247461.8377604,
 33206]

In [48]:
result=[]
pointer1=0
pointer2=0
length = len(a)
for i in range(length):
    print(pointer1,pointer2)
    if a[i]==" ":
        result.append(a[pointer1:i])
        pointer1 = i

   
    

0 0
0 0
0 0
0 0
0 0
0 0
0 0
6 0
6 0
6 0
9 0
9 0
9 0
9 0
13 0
13 0
13 0
13 0
13 0
18 0
18 0
18 0
21 0
21 0
21 0
21 0
25 0
25 0
25 0
25 0
25 0
30 0
31 0
32 0


In [ ]:
result

['rename', ' -f', ' old', ' name', ' -b', ' new', ' name', ' ', ' ', ' ']

In [ ]:
def path_verifier(path):
       return True if self.root.path in path else False
       


In [ ]:
def path_breaker(path):
       rootv=self.root.path.split('/')
       pathv=path.split('/')
       l = len(rootv)
       return pathv[l:]
   
   


In [3]:
def get_node_by_path(self,path):
       if self.path_verifier(path):
           results = self.path_breaker(path)
           root = self.root
           for pathname in results:
               if pathname in root.childs:
                   root = root.childs[pathname]
               elif pathname:
                   return {
                       'result':None,
                       'message':'path not exist'
                   }
             
           return {
                   'result':root,
                   'message':'path exist'
           }
       return {
                   'result':None,
                   'message':'path is incorrect or formate is incorrect'
           }

In [4]:
path_breaker('d:/ssas/sas/as/sa/sas/s')

TypeError: path_breaker() missing 1 required positional argument: 'path'

In [21]:
filename = "uditya(1)"

In [ ]:
import re

def name_ext(filename):
    if '.' in filename:
        parts = filename.split('.')
        _name,_ext = '.'.join(parts[:-1]),parts[-1]
    else:
        _name,_ext = (filename,"unknown")

    return {'name':_name,
        'ext':_ext,
        'isHidden':True if filename.startswith('.') else False}

filename = "udity(uditya)a(100).txt"

def filename_dup_normalizer(filename):
    data = name_ext(filename)
    name = data['name']
    ext = data['ext']

    match = re.search(r"\(\d+\)", name)
    if match:
        pointer = match.group()
        number = re.search(r"\d+",pointer).group()
        new_pointer = f'({int(number) + 1})'
        name = name.replace(pointer,new_pointer)
    else:
        name +='(1)'
    

    return f'{name}' if ext == 'unknown' else f'{name}.{ext}'
      # Output: (1)
filename_dup_normalizer(filename)

'udity(uditya)a(101).txt'

In [19]:
a="-rwxrwxrw-"

In [21]:
filemode_readable(a)

{'owner': ['read', 'write', 'execute'],
 'groups': ['read', 'write', 'execute'],
 'everyone': ['read', 'write', '']}

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 
import pickle
import os
from FS.constant import COSINE_SIMILARITY_MINVALUE


class MrVectorExpert:
    def __init__(self, db_path="save/vectormaster.pkl", autosave=True):
        # model: SentenceTransformer model
        self.model = None
        self.isModelLoaded = False

        self.db_path = db_path
        self.autosave = autosave
        self.path_to_vector = {}

    def load_model(self):
        self.model = SentenceTransformer(r".\model\all-MiniLM-L6-v2")
        self.isModelLoaded = True

    def background_task_step(self):
        if not self.isModelLoaded:
            self.load_model()
            return True
        return False

    # ---------------- Core Operations ---------------- #

    def insert(self, path: str, file_vector: str):
        """
        Insert or update path with vector
        """
        if path in self.path_to_vector:
            old_vector = self.path_to_vector[path]
            if np.all(old_vector == file_vector):
                return  # no change

        self.path_to_vector[path] = file_vector
        if self.autosave:
            self.save()

    def delete(self, path: str):
        """
        Delete path entry
        """
        if path not in self.path_to_vector:
            return

        self.path_to_vector.pop(path)
        logger.info(f"delete {path} in myvector ")

        if self.autosave:
            self.save()
    

    def convert_tags_to_vector(self, tags):
        """
        Convert a list of tags to a single mean vector.
        Args:
            tags: List of tag strings
        Returns:
            numpy array: Mean vector of all tag embeddings
        """
        if not tags:
            return None
        if self.isModelLoaded:
            # Get embeddings for all tags
            embeddings = self.model.encode(tags)
            # Calculate mean vectors
            mean_vector = np.mean(embeddings, axis=0)

            return mean_vector
        return None

    def calculate_cosine_similarity(self, vector1, vector2):
        """
        Calculate cosine similarity between two vectors.
        Args:
            vector1: First vector (numpy array)
            vector2: Second vector (numpy array)
        Returns:
            float: Cosine similarity score (between -1 and 1)
        """
        # Reshape vectors for sklearn
        v1 = vector1.reshape(1, -1)
        v2 = vector2.reshape(1, -1)
        # Calculate cosine similarity
        similarity = cosine_similarity(v1, v2)[0][0]
        return similarity

    def match_tag_sets(self, tags1, tags2):
        """
        Compare two sets of tags and return their similarity score.
        Args:
            tags1: First list of tags
            tags2: Second list of tags
        Returns:
            float: Similarity score between the two tag sets
        """
        vector1 = self.convert_tags_to_vector(tags1)
        vector2 = self.convert_tags_to_vector(tags2)

        if vector1 is None or vector2 is None:
            return 0.0

        similarity = self.calculate_cosine_similarity(vector1, vector2)
        return similarity

        
    # ---------------- Search ---------------- #

    def search_by_vector(self,vector1):
        logger.info("context based search start")
        result = []
        for path,vector2 in self.path_to_vector.items():
            score = self.calculate_cosine_similarity(vector1,vector2)
            if score >= COSINE_SIMILARITY_MINVALUE:
                result.append(path)
        return result

    def search_by_query(self,q):
        vector = self.convert_tags_to_vector([q])
        return self.search_by_vector(vector)


    # ---------------- Persistence ---------------- #

    def save(self):
        """
        Save vectorMaster state to pickle
        """
        data = {
            "path_to_vector": self.path_to_vector
            
        }
        with open(self.db_path, "wb") as f:
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

        logger.info(f"update or save {self.db_path} ")

    def load(self):
        """
        Load vectorMaster state from pickle
        """
        if not os.path.exists(self.db_path):
            logger.error(f'{self.db_path} not exist')
            return

        try:
            with open(self.db_path, "rb") as f:
                data = pickle.load(f)

            self.path_to_vector = data.get("path_to_vector", {})
            logger.info(f"{self.db_path} successfully loaded")

        except Exception:
            # Corrupted pickle → safe reset
            self.path_to_vector = {}
            logger.warning(f"{self.db_path} Corrupted pickle → safe reset")
        



d:\don'tDelete\Desktop\file management system\VE\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

print("Loading model...")
mrvector = MrVectorExpert()
mrvector.load_model()
print("Model loaded successfully\n")


Loading model...
Model loaded successfully



In [5]:

# Example 1: Convert tags to vector
print("=== Example 1: Convert tags to mean vector ===")
tags1 = ["python", "programming", "coding"]
vector1 = mrvector.convert_tags_to_vector(tags1)
print(f"Tags: {tags1}")
print(f"Vector shape: {vector1.shape}")
print(f"Vector (first 5 values): {vector1[:5]}\n")


=== Example 1: Convert tags to mean vector ===
Tags: ['python', 'programming', 'coding']
Vector shape: (384,)
Vector (first 5 values): [-0.0413085   0.02523329 -0.06452059  0.00773893 -0.04015379]



In [6]:
# Example 2: Compare similar tag sets
print("=== Example 2: Compare similar tag sets ===")
tags_set1 = ["python", "programming", "coding"]
tags_set2 = ["python", "development", "software"]
similarity = mrvector.match_tag_sets(tags_set1, tags_set2)
print(f"Tags Set 1: {tags_set1}")
print(f"Tags Set 2: {tags_set2}")
print(f"Similarity Score: {similarity:.4f}\n")


=== Example 2: Compare similar tag sets ===
Tags Set 1: ['python', 'programming', 'coding']
Tags Set 2: ['python', 'development', 'software']
Similarity Score: 0.8214



In [7]:
# Example 3: Compare different tag sets
print("=== Example 3: Compare different tag sets ===")
tags_set3 = ["cooking", "recipe", "food"]
tags_set4 = ["python", "programming", "coding"]
similarity = mrvector.match_tag_sets(tags_set3, tags_set4)
print(f"Tags Set 3: {tags_set3}")
print(f"Tags Set 4: {tags_set4}")
print(f"Similarity Score: {similarity:.4f}\n")


=== Example 3: Compare different tag sets ===
Tags Set 3: ['cooking', 'recipe', 'food']
Tags Set 4: ['python', 'programming', 'coding']
Similarity Score: 0.3294



In [12]:
# Example 4: Compare multiple tag sets
print("=== Example 4: Find most similar tags ===")
query_tags = ["machine learning", "AI", "neural networks","data science"]
# query_tags = ["machine learning"]
candidate_tags = [
    ["deep learning", "artificial intelligence", "ML"],
    ["cooking", "baking", "recipes"],
    ["data science", "statistics", "analytics"],
    ["web development", "HTML", "CSS"],
]
print(f"Query tags: {query_tags}\n")
query_vector = mrvector.convert_tags_to_vector(query_tags)
results = []
for i, candidate in enumerate(candidate_tags):
    candidate_vector = mrvector.convert_tags_to_vector(candidate)
    similarity = mrvector.calculate_cosine_similarity(
        query_vector, candidate_vector
    )
    results.append((i, candidate, similarity))
# Sort by similarity (descending)
results.sort(key=lambda x: x[2], reverse=True)
print("Results (sorted by similarity):")
for rank, (idx, tags, score) in enumerate(results, 1):
    print(f"{rank}. Tags: {tags}")
    print(f"   Similarity: {score:.4f}\n")


=== Example 4: Find most similar tags ===
Query tags: ['machine learning', 'AI', 'neural networks', 'data science']

Results (sorted by similarity):
1. Tags: ['deep learning', 'artificial intelligence', 'ML']
   Similarity: 0.8360

2. Tags: ['data science', 'statistics', 'analytics']
   Similarity: 0.6966

3. Tags: ['web development', 'HTML', 'CSS']
   Similarity: 0.4187

4. Tags: ['cooking', 'baking', 'recipes']
   Similarity: 0.3296



In [9]:
# Example 5: Direct cosine similarity between two vectors
print("=== Example 5: Direct vector comparison ===")
tags_a = ["data", "analytics"]
tags_b = ["statistics", "analysis"]
vector_a = mrvector.convert_tags_to_vector(tags_a)
vector_b = mrvector.convert_tags_to_vector(tags_b)
similarity = mrvector.calculate_cosine_similarity(vector_a, vector_b)
print(f"Tags A: {tags_a}")
print(f"Tags B: {tags_b}")
print(f"Cosine Similarity: {similarity:.4f}")

=== Example 5: Direct vector comparison ===
Tags A: ['data', 'analytics']
Tags B: ['statistics', 'analysis']
Cosine Similarity: 0.7555


In [25]:
a = "d:/don'tDelete/Desktop/file management system/testnew/uditya.txt"

'uditya.txt'

In [ ]:
with open("randomm.txt","a") as f:
    f.write()

In [11]:
import time

In [30]:
class Custom_Log:
    def __init__(self,src="change.log"):
        self.src = src
        
    def put_change_log(self,msg,path,hash_):
        with open(self.src,"a") as f:
                f.write(f'{time.asctime()} | {msg} | {path} | {hash_}\n')
        
        

In [31]:
l = Custom_Log()


In [36]:
l.put_change_log("file change detected","d:/don'tDelete/Desktop/file management system/testnew/uditya.txt","jdhkhdkjhkjhd")

In [13]:
data = {'c814ebbca686d788dee8132a10c405508397addb96b4d7213a3794d25d8d017e': {"d:/don'tDelete/Desktop/file management system/testnew/zzzznew/z.txt", "d:/don'tDelete/Desktop/file management system/testnew/zzzznew/udii/z.txt", "d:/don'tDelete/Desktop/file management system/testnew/zzzz(2)/zsd.txt"}, 'eb6108152dc95f33d27c78166266696b4e3d15a2f062d2751c5fc599f6be6f27': {"d:/don'tDelete/Desktop/file management system/testnew/zzzz(1)/uditya.txt", "d:/don'tDelete/Desktop/file management system/testnew/uditya.txt", "d:/don'tDelete/Desktop/file management system/testnew/zzzz(2)/uditya.txt"}}

In [14]:
hash_ = 'c814ebbca686d788dee8132a10c405508397addb96b4d7213a3794d25d8d017e'
paths = {"d:/don'tDelete/Desktop/file management system/testnew/zzzznew/z.txt", "d:/don'tDelete/Desktop/file management system/testnew/zzzznew/udii/z.txt", "d:/don'tDelete/Desktop/file management system/testnew/zzzz(2)/zsd.txt"}

In [18]:
result = []
for hash_,paths in data.items():
    result.append(f'hash:{hash_}\n   |--->{'\n   |--->'.join(paths)}')



In [19]:
for i in result:
    print(i)
    print()

hash:c814ebbca686d788dee8132a10c405508397addb96b4d7213a3794d25d8d017e
   |--->d:/don'tDelete/Desktop/file management system/testnew/zzzznew/udii/z.txt
   |--->d:/don'tDelete/Desktop/file management system/testnew/zzzz(2)/zsd.txt
   |--->d:/don'tDelete/Desktop/file management system/testnew/zzzznew/z.txt

hash:eb6108152dc95f33d27c78166266696b4e3d15a2f062d2751c5fc599f6be6f27
   |--->d:/don'tDelete/Desktop/file management system/testnew/zzzz(2)/uditya.txt
   |--->d:/don'tDelete/Desktop/file management system/testnew/zzzz(1)/uditya.txt
   |--->d:/don'tDelete/Desktop/file management system/testnew/uditya.txt



In [ ]:
print(f'--->{'\n--->'.join(paths)}' if paths else 'not duplicate found')

not duplicate found


In [25]:
paths={}

In [11]:
a= "   ls s  "

In [12]:
a.split()[0]

'ls'

In [18]:
from asyncio.selector_events import base_events


def h(self,command):
        base = command.split()[0]
        
        commands = {
            "ls":0,
            "cd":0,
            "back":0,
            "root":0,
            "refresh":0,
            "select":0,
            "unselect":0,
            "open":0,
            "cut":0,
            "copy":0,
            "del":0,
            "mkdir":0,
            "mkf":0,
            "write":0,
            "append":0,
            "info":0,
            "rename":0,
            "tree":0,
            "search":0,
            "find":0,
            "exit":0,
            "pwd":0,
            "tag":0,
            "verify":0,
            "status":0
        }
        return commands.get(base,lambda:f'{base} not found as internall command')()

In [7]:
c,name = 'cd ./'.strip().split(" ",1)
c,name

('cd', './')

In [5]:
'ls'.split()

['ls']

In [8]:
nano1 = time.perf_counter_ns()
nano2 = time.perf_counter_ns()

In [9]:
nano2-nano1

26800

In [1]:
from queue import Queue

In [2]:
q= Queue()

AttributeError: 'Queue' object has no attribute '__len__'